# Group Lab 2


## Scoring Guide:
- Total Points Available: 20 points
    - 5 points: • Demonstrated and applied at least one statistical method to analyze variable correlation, validity of data, time-series
                   trend, or other method, OR developed a descriptive, predictive, or prescriptive data model.
                • Included rough draft of 2-3 (two to three) data visualizations to be used in final capstone presentation
    - 5 points: Presentation of the project given to class was well organized, and DID NOT EXCEED ten (10) minutes

In [244]:
#Team 1 Mchmil, JFlores, LMcHenry, LRyan, CYonley
#Problem statement
#Why do certain areas have higher land value compared to building value? (compare price per acre or price per square foot)

In [245]:
#load necessary packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from math import exp

In [246]:
%matplotlib inline
pd.set_option('display.max_column', 200)
plt.figure(num=None, figsize=(12, 10), dpi=80, facecolor='w', edgecolor='k')
sns.set(rc={'figure.figsize': (12, 10), "lines.markeredgewidth": 0.5 })

<Figure size 960x800 with 0 Axes>

In [247]:
# load franklin data
franklin_df = pd.read_csv('home1_data.csv', low_memory=False)
# rename columns to match charlotte data
franklin_df.rename(columns={'AppraisedTaxableLand': 'Land_Value', 'AppraisedTaxableBuilding': 'Building_Value', 'YearBuilt': 'Year_Built', \
                            'USPS_CITY': 'City', 'County': 'CountyMunicipality', 'DESCR1': 'Address'}, inplace=True)
franklin_df.shape
#create sample of dataset
franklin_sample = franklin_df.sample(frac=0.2, random_state=1)
print(len(franklin_df))
print(len(franklin_sample))
franklin_sample.to_csv("franklin_sample.csv")

35264
7053


In [ ]:
# load north carolina data
nc_df = pd.read_csv('Parcel_Look_Up.csv', low_memory=False)
nc_df.rename(columns={'Building_Type': 'PropertyClass', 'Tax_Fire_District': 'TAXDESI', 'Municipality': 'CountyMunicipality',\
                      'Heated_Sqft': 'Area', 'Total_Acreage': 'Acreage', 'Mailing_Address': 'Address'}, inplace=True)
# filter for residential data to match franklin
charlotte_df = nc_df[nc_df.PropertyClass == 'RES']
charlotte_df.shape
#create sample of dataset
charlotte_sample = charlotte_df.sample(frac=0.2, random_state=1)
print(len(charlotte_df))
print(len(charlotte_sample))
charlotte_sample.to_csv("charlotte_sample.csv")

In [ ]:
#append both datasets
csv_files = ['franklin_sample.csv', 'charlotte_sample.csv']
franklin_charlotte_data = pd.DataFrame()

# append the CSV files
for file in csv_files:
    df = pd.read_csv(file, low_memory=False)
    franklin_charlotte_data = franklin_charlotte_data.append(df, ignore_index=True)

franklin_charlotte_data.head()

In [ ]:
franklin_charlotte_data.to_csv("master.csv")

In [ ]:
# clean data so values match
franklin_charlotte_data2 = franklin_charlotte_data.copy()
franklin_charlotte_data2.PropertyClass.replace(r'(RES)', value='R', regex=True, inplace=True)
franklin_charlotte_data2['CountyMunicipality']=franklin_charlotte_data2['CountyMunicipality'].apply(str.upper)
franklin_charlotte_data2.head()

In [ ]:
franklin_charlotte_data2.PropertyClass.value_counts(normalize=True)
#sns.countplot(y=franklin_charlotte_data2.PropertyClass)

In [ ]:
franklin_charlotte_data2.CountyMunicipality.value_counts(normalize=True)
#Lab 1 Data Visualization
#sns.countplot(y=franklin_charlotte_data2.CountyMunicipality)

In [ ]:
franklin_charlotte_data2.Land_Value.describe()

In [ ]:
franklin_charlotte_data2.Building_Value.describe()

In [ ]:
#Lab 1 Data Visualization
#x = franklin_charlotte_data2.Building_Value
#sns.displot(x, kde=False, bins=40)

In [ ]:
#Lab 1 Data Visualization
#x = franklin_charlotte_data2.Land_Value
#sns.displot(x, kde=False, bins=40)

In [ ]:
#Lab 1 Data Visualization
#franklin_charlotte_data2.hist()

In [ ]:
#Lab 1 Data Visualization
#franklin_charlotte_data2.plot.scatter(x="Land_Value", y="Building_Value", c='green', alpha=0.2)

In [ ]:
#Lab 1 Data Visualization
#columns = ["Land_Value", "Building_Value", "Year_Built"]
#sns.pairplot(franklin_charlotte_data2[columns])

In [ ]:
#Lab 1 Data Visualization
#franklin_charlotte_data2.CountyMunicipality.value_counts().plot(kind="bar")

In [ ]:
franklin_charlotte_data2.round(8).head()

In [ ]:
#only look at data where land >= building 
mask = franklin_charlotte_data2.Building_Value <= franklin_charlotte_data2.Land_Value

In [ ]:
franklin_charlotte_data2[mask].head()

In [ ]:
#new data frame of greater land values
greaterland_df = franklin_charlotte_data2.loc[mask]

In [ ]:
greaterland_df.CountyMunicipality.unique()

In [ ]:
greaterland_df.columns.tolist()

In [ ]:
greaterland_df_columns = ['PID', 'Land_Value', 'Building_Value', 'SchoolDistrict', 'CodeDist', 
                     'PropertyClass', 'Price', 'Total_Value', 'Acreage', 'Address', 'Area', 'Year_Built', 'CountyMunicipality']

greaterland_subset = greaterland_df[greaterland_df_columns].copy()

In [ ]:
greaterland_subset.columns.tolist()
greaterland_subset.rename({'Area':'Square_Footage'}, axis=1, inplace=True)

In [ ]:
greaterland_subset.CountyMunicipality.unique()

In [ ]:
greaterland_subset.CountyMunicipality.value_counts()
#Charlotte makes up 82% of data where landvalue >= Buildingvalue

In [ ]:
# Histogram countplot for the properties where land value > building value by municipality.
sns.countplot(y=greaterland_subset.CountyMunicipality)

In [ ]:
# Pair plot for the properties where land value > building value by municipality.
columns = ["Land_Value", "Building_Value", "Year_Built", "CountyMunicipality"]
sns.pairplot(greaterland_subset[columns], hue = "CountyMunicipality")

In [ ]:
greaterland_subset.head()

In [ ]:
#drop rows where address is nan; commenting out due to no nan values
#greaterland_subset2 = greaterland_subset[greaterland_subset['Address'].notna()]

In [ ]:
# Add column for Land Value per Acre and omit rows where Acreage = 0.0
greaterland_subset["Land_Value_per_Acre"] = greaterland_subset["Land_Value"] // greaterland_subset["Acreage"]
mask = greaterland_subset.Acreage > 0.0000
greaterland_subset=greaterland_subset.loc[mask]

# Add column for Building Value per Sq. Ft.
greaterland_subset["Building_Value_per_SqFt"] = greaterland_subset["Building_Value"] // greaterland_subset["Square_Footage"]
greaterland_subset.head()


In [ ]:
# Scatter plot of relative land and building values.
greaterland_subset.plot.scatter(x="Land_Value_per_Acre", y="Building_Value_per_SqFt", c='green', alpha=.5)

In [ ]:
# 25% sample of parcels where land Value >= Building Value
greaterland_subset_sample = greaterland_subset.sample(frac=0.20, random_state=1)
print(len(greaterland_subset))
print(len(greaterland_subset_sample))
greaterland_subset_sample

In [ ]:
# Histogram countplot for the sample value by municipality.
sns.countplot(y=greaterland_subset_sample.CountyMunicipality)

In [ ]:
#import mecklenburg auditor waterfront property data
nc_waterfront = pd.read_csv('mecklenburg_propertyresults-water.csv', low_memory=False)

In [ ]:
#match columns parcel id= parcel
nc_waterfront.rename({'Parcel':'PID'}, axis=1, inplace=True)
nc_waterfront['PID']= nc_waterfront['PID'].replace('-', '', regex=True).astype(int)
greaterland_subset_sample['PID']=greaterland_subset_sample['PID'].replace('-', '', regex=True).astype(int)

In [ ]:
#join by ParcelID greaterland_subset_sample
greaterland_subset_sample.merge(nc_waterfront, on ='PID', how='left')
print(len(greaterland_subset_sample))

In [ ]:
greaterland_subset_sample

In [ ]:
greaterland_subset_sample_t1 = greaterland_subset_sample.copy()

In [ ]:
greaterland_subset_sample_t1.corr()


In [ ]:
#from scipy import stats
#pred_vars = ['Total_Value', \
             #'Land_Value_per_Acre','Building_Value_per_SqFt']
#print(f"Skew:\n{greaterland_subset_sample_t1[pred_vars].skew()}\n\n")
#print(f"Kurtosis:\n{greaterland_subset_sample_t1[pred_vars].kurtosis()}\n\n")
#sns.set(font_scale=1.0)
#display(greaterland_subset_sample_t1[pred_vars].hist(bins=80))
greaterland_subset_sample_t1.replace([np.inf, -np.inf], 0, inplace=True)
from scipy import stats
greaterland_subset_sample_t1['Building_Value_per_SqFt'] = greaterland_subset_sample_t1['Building_Value_per_SqFt'].fillna(0)
greaterland_subset_sample_t1['Building_Value_per_SqFt'] = greaterland_subset_sample_t1['Building_Value_per_SqFt'].copy()
pred_vars = ['Total_Value',\
'Land_Value_per_Acre','Building_Value_per_SqFt']
print(f"Skew:\n{greaterland_subset_sample_t1[pred_vars].skew()}\n\n")
print(f"Kurtosis:\n{greaterland_subset_sample_t1[pred_vars].kurtosis()}\n\n")
sns.set(font_scale=1.0)
display(greaterland_subset_sample_t1[pred_vars].hist(bins=80))
#plt.hist(greaterland_subset_sample_t1[pred_vars], range=[0, 100000])

In [ ]:
from scipy.stats import boxcox
from scipy import special
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf

pd.plotting.register_matplotlib_converters()
%matplotlib inline
pd.set_option('display.max_column', 200)
plt.figure(num=None, figsize=(12, 10), dpi=80, facecolor='w', edgecolor='k')
sns.set(rc={'figure.figsize': (12, 10), "lines.markeredgewidth": 0.5 })



print(greaterland_subset_sample_t1)
train, test = train_test_split(greaterland_subset_sample_t1, test_size=.5, random_state=0, shuffle=True)

In [ ]:
#OLS for Total_Value
f_rev = 'Total_Value ~ Land_Value_per_Acre + Building_Value_per_SqFt'
model = smf.ols(formula=f_rev, data=train).fit()
model.summary()

In [ ]:
greaterland_subset_sample_t1.to_csv("Updatedsample2.csv")